<a href="https://colab.research.google.com/github/dewipuspita0904/Proyek-Analisis-Sentimen-Awal/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persiapan

## Import Library

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
from keras.layers import Bidirectional
from keras.optimizers import AdamW
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string

## Load Dataset Hasil Scraping

In [47]:
df = pd.read_csv('dataset_playstore_reviews.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,863c2e30-d9fc-48db-bfb1-6a4571ea60ca,Ai teuis Sumiyati 94,https://play-lh.googleusercontent.com/a-/ALV-U...,kenapa gs bisa menginstal,5,0,NaN,2025-04-06 06:24:35,NaN,NaN,NaN
1,7491182c-bcf1-45d6-b76c-e793290d4b69,Maria Goreti,https://play-lh.googleusercontent.com/a/ACg8oc...,baik,3,0,NaN,2025-04-06 03:00:03,NaN,NaN,NaN
2,0a16ca0e-8aab-4b2b-852c-5c4136d24c3e,Rahma,https://play-lh.googleusercontent.com/a/ACg8oc...,bagus,5,0,NaN,2025-04-06 00:02:39,NaN,NaN,NaN
3,646bfbb1-befc-4297-ae2d-41c32947b7a9,Tandi Saputra,https://play-lh.googleusercontent.com/a-/ALV-U...,saya mengalami kendala dibagian showcase padah...,1,605,39.3.3,2025-04-05 15:24:06,NaN,NaN,39.3.3
4,3fea6d2f-a0f5-4cc6-9b30-1bc70cb9aa1e,Ahmad Sodik,https://play-lh.googleusercontent.com/a/ACg8oc...,pokony oke,5,0,39.3.3,2025-04-05 11:29:48,NaN,NaN,39.3.3


In [48]:
print("Jumlah data:", len(df))
df.head()
df['score'].value_counts()

Jumlah data: 10000


,count
score,
5,7328
1,1142
4,727
3,502
2,301


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               10000 non-null  object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  6002 non-null   object
 7   at                    10000 non-null  object
 8   replyContent          1 non-null      object
 9   repliedAt             1 non-null      object
 10  appVersion            6002 non-null   object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


In [50]:
df.drop(['reviewCreatedVersion','replyContent', 'repliedAt', 'appVersion'], axis=1, inplace=True)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,at
0,863c2e30-d9fc-48db-bfb1-6a4571ea60ca,Ai teuis Sumiyati 94,https://play-lh.googleusercontent.com/a-/ALV-U...,kenapa gs bisa menginstal,5,0,2025-04-06 06:24:35
1,7491182c-bcf1-45d6-b76c-e793290d4b69,Maria Goreti,https://play-lh.googleusercontent.com/a/ACg8oc...,baik,3,0,2025-04-06 03:00:03
2,0a16ca0e-8aab-4b2b-852c-5c4136d24c3e,Rahma,https://play-lh.googleusercontent.com/a/ACg8oc...,bagus,5,0,2025-04-06 00:02:39
3,646bfbb1-befc-4297-ae2d-41c32947b7a9,Tandi Saputra,https://play-lh.googleusercontent.com/a-/ALV-U...,saya mengalami kendala dibagian showcase padah...,1,605,2025-04-05 15:24:06
4,3fea6d2f-a0f5-4cc6-9b30-1bc70cb9aa1e,Ahmad Sodik,https://play-lh.googleusercontent.com/a/ACg8oc...,pokony oke,5,0,2025-04-05 11:29:48


In [51]:
df.duplicated().sum()

np.int64(0)

In [52]:
df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
at,0


In [53]:
df_cleaned = df.dropna()
print(f"Data Isnull : {df_cleaned.isnull().sum().sum()}")
print(f"Data Duplicated : {df_cleaned.duplicated().sum().sum()}")
df_cleaned.info()

Data Isnull : 0
Data Duplicated : 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reviewId       10000 non-null  object
 1   userName       10000 non-null  object
 2   userImage      10000 non-null  object
 3   content        10000 non-null  object
 4   score          10000 non-null  int64 
 5   thumbsUpCount  10000 non-null  int64 
 6   at             10000 non-null  object
dtypes: int64(2), object(5)
memory usage: 547.0+ KB


In [54]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['cleaned_comment'] = df['content'].apply(clean_text)

## Labeling

In [55]:
def label_sentiment(text):
    from textblob import TextBlob
    polarity = TextBlob(text).sentiment.polarity
    if polarity < -0.1:
        return 'negative'
    elif polarity <= 0.1:
        return 'neutral'
    else:
        return 'positive'

df['label'] = df['cleaned_comment'].apply(label_sentiment)

## Data Preparation

In [56]:
y = LabelEncoder().fit_transform(df['label'])
X = df['cleaned_comment']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modeling

### SVM - TF-IDF

In [57]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Jumlah data asli:", Counter(y_train))
print("Jumlah data setelah SMOTE:", Counter(y_train_resampled))

Jumlah data asli: Counter({np.int64(1): 6699, np.int64(2): 1262, np.int64(0): 39})
Jumlah data setelah SMOTE: Counter({np.int64(1): 6699, np.int64(2): 6699, np.int64(0): 6699})


In [58]:
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_resampled, y_train_resampled)

y_pred_svm = svm.predict(X_test)
accuracy_svm = svm.score(X_test, y_test)

print("Accuracy:", accuracy_svm)
print(classification_report(y_test, y_pred_svm))

Accuracy: 0.98
              precision    recall  f1-score   support

           0       0.86      0.60      0.71        10
           1       0.99      0.99      0.99      1675
           2       0.95      0.94      0.94       315

    accuracy                           0.98      2000
   macro avg       0.93      0.84      0.88      2000
weighted avg       0.98      0.98      0.98      2000



### RF - Word2Vec

In [59]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_resampled, y_train_resampled)

y_pred_rf = rf.predict(X_test)
accuracy_rf = rf.score(X_test, y_test)

print("Accuracy:", accuracy_rf)
print(classification_report(y_test, y_pred_rf))

Accuracy: 0.975
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        10
           1       0.98      0.99      0.99      1675
           2       0.96      0.90      0.93       315

    accuracy                           0.97      2000
   macro avg       0.98      0.76      0.83      2000
weighted avg       0.97      0.97      0.97      2000



### DL LSTM

In [60]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train_raw)

X_train_seq = tokenizer.texts_to_sequences(X_train_raw)
X_test_seq = tokenizer.texts_to_sequences(X_test_raw)

maxlen = 200
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

y_train_cat = pd.get_dummies(y_train).values
y_test_cat = pd.get_dummies(y_test).values

model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=100, input_length=maxlen),
    Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer='l2')),
    Dropout(0.5),
    Bidirectional(LSTM(64, kernel_regularizer='l2')),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-4),
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]

history = model_lstm.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_test_pad, y_test_cat),
    epochs=15,
    batch_size=64,
    callbacks=callbacks
)

loss, accuracy_lstm = model_lstm.evaluate(X_test_pad, y_test_cat)

print(accuracy_lstm)

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 130s 996ms/step - accuracy: 0.8100 - loss: 4.8749 - val_accuracy: 0.8375 - val_loss: 0.5580 - learning_rate: 0.0010
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 141s 987ms/step - accuracy: 0.8425 - loss: 0.5159 - val_accuracy: 0.8375 - val_loss: 0.4710 - learning_rate: 0.0010
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 121s 971ms/step - accuracy: 0.8401 - loss: 0.4759 - val_accuracy: 0.8375 - val_loss: 0.4709 - learning_rate: 0.0010
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 972ms/step - accuracy: 0.8467 - loss: 0.4588 - val_accuracy: 0.8375 - val_loss: 0.4666 - learning_rate: 0.0010
Epoch 5/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 124s 991ms/step - accuracy: 0.8403 - loss: 0.4777 - val_accuracy: 0.8375 - val_loss: 0.4690 - learning_rate: 0.0010
Epoch 6/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 123s 989ms/step - accuracy: 0.8383 - loss: 0.4743 - val_accuracy: 0.8375 - val_loss: 0.4681 - learning_rate: 0.0010
Epoch 7/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 991ms/step - accuracy: 0.8315 

## Inference

In [61]:
accuracy = {'SVM': accuracy_svm, 'Random Forest': accuracy_rf, 'LSTM': accuracy_lstm}
best_model = max(accuracy, key=accuracy.get)
print(f"Model terbaik adalah {best_model} dengan akurasi {accuracy[best_model]}")

Model terbaik adalah SVM dengan akurasi 0.98


In [62]:
le = LabelEncoder()
le.fit(df['label'])

if best_model == 'SVM':
    model = svm
    prediction = y_pred_svm
elif best_model == 'Random Forest':
    model = rf
    prediction = y_pred_rf
elif best_model == 'LSTM':
    model = model_lstm
    prediction = model_lstm.predict(X_test_pad)
    prediction = np.argmax(prediction, axis=1)

comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': prediction})
comparison_df['Actual'] = le.inverse_transform(comparison_df['Actual'])
comparison_df['Predicted'] = le.inverse_transform(comparison_df['Predicted'])

print("Hasil Prediksi:")
print(comparison_df.head())

Hasil Prediksi:
    Actual Predicted
0  neutral   neutral
1  neutral   neutral
2  neutral   neutral
3  neutral   neutral
4  neutral   neutral


In [63]:
pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.5.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.21.0
astropy==7.0.1
astropy-iers-data==0.2025.3.31.0.36.18
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.42.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.2.1
blosc2==3.2.1
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
